#  Spark RDD and Beyond

## Types of RDD

- ParallelCollectionRDD — An RDD created by Spark Context through parallelizing an existing collection. Eg. sc.parallelize, sc.makeRDD
- CoGroupedRDD — An RDD that cogroups its parents. For each key k in parent RDDs, the resulting RDD contains a tuple with the list of values for that key.
- HadoopRDD is an RDD that provides core functionality for reading data stored in HDFS using the older MapReduce API. The most notable use case is the return RDD of SparkContext.textFile.
- MapPartitionsRDD — An RDD that applies the provided function to every partition of the parent RDD. A result of calling operations like map, flatMap, filter, mapPartitions, etc.
- CoalescedRDD — a result of repartition or coalesce transformations.
- ShuffledRDD — a result of shuffling, e.g. after repartition or coalesce transformations.
- PipedRDD — an RDD created by piping elements to a forked external process.
- SequenceFileRDD — is an RDD that can be saved as a SequenceFile.

https://medium.com/knoldus/rdd-sparks-fault-tolerant-in-memory-weapon-130f8df2f996

# Let's do practice

In [ ]:
import findspark
import pyspark
findspark.find() 
findspark

In [ ]:
conf = pyspark.SparkConf().setAppName('Tap').setMaster('local')
sc = pyspark.SparkContext(conf=conf)
sc

## Create a RDD using parallelize

In [ ]:
# Create a RDD 
digits = sc.parallelize([0,1,2,3,4,5,6,7,8,9])
digits

**RDD.collect()**

Return a list that contains all of the elements in this RDD.

**Notes**

This method should only be used if the resulting array is expected to be small, as all the data is loaded into the driver’s memory.

In [ ]:
digits.collect()

[▶![](http://img.youtube.com/vi/MVkjC-doVP4/0.jpg)](https://www.youtube.com/watch?v=MVkjC-doVP4&start=120)

## RDD Operations

RDDs support two types of operations: 
- **transformations**, which create a new dataset from an existing one
- **actions**, which return a value to the driver program after running a computation on the dataset. 

For example, *map* is a transformation that passes each dataset element through a function and returns a new RDD representing the results.

On the other hand, *reduce* is an action that aggregates all the elements of the RDD using some function and returns the final result to the driver program (although there is also a parallel reduceByKey that returns a distributed dataset).

All transformations in Spark are **lazy**, in that they do not compute their results right away. Instead, they just remember the transformations applied to some base dataset (e.g. a file). 

he transformations are only computed when an action requires a result to be returned to the driver program. 

This design enables Spark to run more **efficiently** . For example, we can realize that a dataset created through map will be used in a reduce and return only the result of the reduce to the driver, rather than the larger mapped dataset.

![](https://media.licdn.com/dms/image/C5612AQHZTuPnWQT6KQ/article-cover_image-shrink_600_2000/0/1520128748970?e=2147483647&v=beta&t=lJDV7DWofBg1ijqPLYGbGDr-FVX_pM6nks-RWQQgRZo)

By default, each transformed RDD may be recomputed each time you run an action on it. However, you may also persist an RDD in memory using the persist (or cache) method, in which case Spark will keep the elements around on the cluster for much faster access the next time you query it. There is also support for persisting RDDs on disk, or replicated across multiple nodes.

![](https://cdn.analyticsvidhya.com/wp-content/uploads/2020/11/persist.jpg)

https://www.analyticsvidhya.com/blog/2020/11/8-must-know-spark-optimization-tips-for-data-engineering-beginners/

# Trasformation
https://spark.apache.org/docs/latest/rdd-programming-guide.html#transformations

![](https://i.ytimg.com/vi/2IQxywxlMO8/hqdefault.jpg)

## map(func)	
Return a new distributed dataset formed by passing each element of the source through a function func. 

In [ ]:
squares=digits.map(lambda x: x*x)

In [ ]:
squares

In [ ]:
squares.collect()

## filter(func)	
Return a new dataset formed by selecting those elements of the source on which func returns true.

In [ ]:
evens = digits.filter(lambda x: x % 2 == 0)

In [ ]:
evens.collect()

## flatMap(func)	
Similar to map, but each input item can be mapped to 0 or more output items (so func should return a Seq rather than a single item).

In [ ]:
def factors(nr):
    i = 2
    factors = []
    while i <= nr:
        if (nr % i) == 0:
            factors.append(i)
            nr = nr / i
        else:
            i = i + 1
    return factors
primes = digits.flatMap(factors)
primes.collect()

## distinct()
Return a new RDD that contains the distinct elements of a RDD.

In [ ]:
primes=primes.distinct()
primes.collect()

 ## sample()
Return a sampled subset of this RDD.

Parameters
- withReplacement – can elements be sampled multiple times (replaced when sampled out)
- fraction – expected size of the sample as a fraction of this RDD’s size without replacement: probability that each element is chosen; fraction must be [0, 1] with replacement: expected number of times each element is chosen; fraction must be >= 0
- seed – seed for the random number generator

In [ ]:
digits.sample(False,0.2).collect()

## union()
Return a new dataset that contains the union of the elements in the source dataset and the argument.

In [ ]:
odds = digits.filter(lambda x: x % 2 == 1)
yy = evens.union(odds)
yy.collect()

## intersection()
Return a new RDD that contains the intersection of elements in the source dataset and the argument.

In [ ]:
intersects=evens.intersection(squares)
intersects.collect()

## cartesian()
When called on datasets of types T and U, returns a dataset of (T, U) pairs (all pairs of elements).

In [ ]:
cart=evens.cartesian(odds)
cart.collect()

## groupByKey()
When called on a dataset of (K, V) pairs, returns a dataset of (K, Iterable<V>) pairs.

In [ ]:
cartgroup=cart.groupByKey()
cartgroup.collect()

![](https://media.makeameme.org/created/nice-235c1c3f2e.jpg)
[Source](https://makeameme.org/meme/nice-235c1c3f2e)

Let's do something better

In [ ]:
cartgroup.map(lambda x : (x[0], list(x[1]))).collect()

## join(otherDataset, [numPartitions])
When called on datasets of type (K, V) and (K, W), returns a dataset of (K, (V, W)) pairs with all pairs of elements for each key. Outer joins are supported through leftOuterJoin, rightOuterJoin, and fullOuterJoin.

In [ ]:
cart2=squares.cartesian(odds)
cart2.collect()

In [ ]:
cart.join(cart2).collect()

## cogroup(otherDataset, [numPartitions])
When called on datasets of type (K, V) and (K, W), returns a dataset of (K, (Iterable<V>, Iterable<W>)) tuples. This operation is also called groupWith

In [ ]:
cart.cogroup(cart2).map(lambda x : (x[0], list(x[1][0])+list(x[1][1]))).collect()

## pipe(command, [envVars])
Pipe each partition of the RDD through a shell command, e.g. a Perl or bash script. RDD elements are written to the process's stdin and lines output to its stdout are returned as an RDD of strings.


In [ ]:
digits.pipe("tee").collect()

# Actions
https://spark.apache.org/docs/latest/rdd-programming-guide.html#actions

![](https://i.imgflip.com/315qmr.jpg)
[Source](https://imgflip.com/i/315qmr)

## reduce(func)
Aggregate the elements of the dataset using a function func (which takes two arguments and returns one). 

The function should be commutative and associative so that it can be computed correctly in parallel.


In [ ]:
sumOfDigits=digits.reduce(lambda a,b:a+b)
sumOfDigits

## count()
Return the number of elements in the dataset.

In [ ]:
digits.count()

## collect()
Return all the elements of the dataset as an array at the driver program. This is usually useful after a filter or other operation that returns a sufficiently small subset of the data.

In [ ]:
cart.collect()

## take(n)

In [ ]:
digits.take(2)

## takeSample(withReplacement, num, [seed])
Return an array with a random sample of num elements of the dataset, with or without replacement, optionally pre-specifying a random number generator seed.

In [ ]:
digits.takeSample(True,5)

## takeOrdered(n, [ordering])
Return the first n elements of the RDD using either their natural order or a custom comparator.


In [ ]:
digits.takeOrdered(5)

## first()
Return the first element of the dataset (similar to take(1)).

In [ ]:
digits.first()

## countByKey()
Write the elements of the dataset in a simple format using Java serialization, which can then be loaded using SparkContext.objectFile().

In [ ]:
cart.countByKey()

## histogram(buckets)
Compute a histogram using the provided buckets. The buckets are all open to the right except for the last which is closed. e.g. [1,10,20,50] means the buckets are [1,10) [10,20) [20,50], which means 1<=x<10, 10<=x<20, 20<=x<=50. And on the input of 1 and 50 we would have a histogram of 1,0,1.

In [ ]:
rdd = sc.parallelize(range(51))

In [ ]:
rdd.histogram(2)

In [ ]:
rdd.histogram([0, 5, 25, 50])

In [ ]:
digits.collect()

In [ ]:
# Works also with strings
rdd = sc.parallelize(["ab", "ac", "b", "bd", "ef"])
rdd.histogram(("a", "b", "c"))

## stats()
Return a StatCounter object that captures the mean, variance and count of the RDD’s elements in one operation.

In [ ]:
digits.stats()

## foreach(func)
Run a function func on each element of the dataset. This is usually done for side effects such as updating an Accumulator or interacting with external storage systems. 
Note: modifying variables other than Accumulators outside of the foreach() may result in undefined behavior. See Understanding closures for more details.

In [ ]:
countdown=sc.parallelize(range(11))
countdown.foreach(lambda x: print("[%-10s] %d%%" % ('='*x, 10*x)))

In [ ]:
sc.stop()